# RAG assisted Auto Developer 2 - nested sessions
-- with LionAGI, LlamaIndex, Autogen and OAI code interpreter


In [1]:
# %pip install lionagi llama_index pyautogen

In [2]:
from pathlib import Path
import lionagi as li

In [3]:
ext=".py"                               # extension of files of interest, can be str or list[str]
data_dir = Path.cwd() / 'lionagi_data'       # directory of source data - lionagi codebase
project_name = "autodev_lion"           # give a project name
output_dir = "data/log/coder/"          # output dir
num=10                                  # max tool use per API call

# A. Use Code QA bot and Code interpreter

### 1. Create query engine

In [4]:
import lionagi as li 

files = li.dir_to_files(dir=data_dir, ext=ext, clean=True, recursive=True,
                        project=project_name, to_csv=True)

chunks = li.file_to_chunks(files, chunk_size=800,  overlap=0.1, 
                           threshold=100, to_csv=True, project=project_name)

20 logs saved to data/logs/sources/autodev_lion_sources_2023-12-21T14_19_24_494075.csv
149 logs saved to data/logs/sources/autodev_lion_sources_2023-12-21T14_19_24_495939.csv


In [8]:
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.schema import TextNode

# build nodes from our existing chunks
f = lambda content: TextNode(text=content)
nodes = li.l_call(chunks, lambda x: f(x["chunk_content"]))

# set up vector index
llm = OpenAI(temperature=0.1, model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)
index1 = VectorStoreIndex(nodes, include_embeddings=True, service_context=service_context)

# set up query engine
query_engine = index1.as_query_engine(include_text=True, response_mode="tree_summarize")

In [9]:
response = query_engine.query("Think step by step, explain how session works in details.")

In [10]:
from IPython.display import Markdown
Markdown(response.response)

The `Session` class is designed to represent a conversation session with a conversational AI system. It orchestrates the flow of the conversation, manages system settings, and interfaces with external tools. Here's a step-by-step explanation of how the `Session` class works:

1. **Initialization**: When a `Session` object is created, it initializes its attributes. This includes creating an instance of the `Conversation` class, which is assigned to the `conversation` attribute. The `system` attribute is set to the current system setting for the conversation, which could define the behavior or configuration of the conversational AI.

2. **Conversation Management**: The `Conversation` instance within the `Session` is responsible for managing the messages that are part of the conversation. This includes storing messages, processing user inputs, and generating responses.

3. **System Settings**: The `system` attribute holds information about the current system settings. These settings could affect how the conversation is conducted, such as which language model to use or specific parameters that tailor the conversation's behavior.

4. **External Interactions**: The `Session` class interacts with external tools and services. This could involve logging data using `DataLogger`, tracking the status of the conversation with `StatusTracker`, or utilizing `ToolManager` to integrate additional functionalities.

5. **Service Communication**: The `Session` may communicate with external services such as `OpenAIService`, which could be an interface to a language model or AI service. This service could be configured using settings from `oai_llmconfig`.

6. **Asynchronous Operations**: The presence of `aiohttp` and `asyncio` in the imports suggests that the `Session` may perform asynchronous HTTP requests, possibly to interact with web-based APIs or services without blocking the execution of the program.

7. **Utility Functions**: The `Session` can use utility functions like `to_list`, `l_call`, and `al_call` from `sys_util`, which might help in list conversion and asynchronous function calling.

8. **API Utilities**: The `Session` might use `api_util` for tracking the status of API calls, ensuring that the conversation session can handle and report on the status of its interactions with external APIs.

9. **Running the Session**: During the conversation, the `Session` would use its methods to receive input, process it, and provide output. This involves using the `Conversation` instance to handle the back-and-forth of messages.

10. **Ending the Session**: Once the conversation is concluded, the `Session` would perform any necessary cleanup, such as logging the conversation's end, updating the status tracker, and closing any external connections if needed.

Throughout its lifecycle, the `Session` acts as the central coordinator for the conversational AI, ensuring that all components work together to provide a seamless conversational experience.

In [11]:
print(response.get_formatted_sources())

> Source (Doc id: 0b0cc526-8bf8-44df-b59b-0739fbe551f9): import aiohttp import asyncio import json from typing import Any import lionagi from .conversatio...

> Source (Doc id: 92686964-a0e9-4da5-87ce-abb61f95aa71): from .message import Message from .conversation import Conversation from .session import Session ...


### 2. Using oai assistant Code Interpreter with Autogen

In [12]:
coder_instruction = f"""
        You are an expert at writing python codes. Write pure python codes, and run it to validate the 
        codes, then return with the full implementation + the word TERMINATE when the task is solved 
        and there is no problem. Reply FAILED if you cannot solve the problem.
        """

In [13]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-4", "gpt4", "gpt-4-32k", "gpt-4-turbo"],
    },
)
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.agentchat import UserProxyAgent

# Initiate an agent equipped with code interpreter
gpt_assistant = GPTAssistantAgent(
    name="Coder Assistant",
    llm_config={
        "tools": [
            {
                "type": "code_interpreter"
            }
        ],
        "config_list": config_list,
    },
    instructions=coder_instruction,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
    human_input_mode="NEVER"
)

async def code_pure_python(instruction):
    user_proxy.initiate_chat(gpt_assistant, message=instruction)
    return gpt_assistant.last_message()

### 3. tools and prompts

In [14]:
tool1 = [
    {
        "type": "function",
        "function": {
            "name": "query_lionagi_codebase",
            "description": """
                Perform a query to a QA bot with access to a vector index 
                built with package lionagi codebase
                """,
            "parameters": {
                "type": "object",
                "properties": {
                    "str_or_query_bundle": {
                        "type": "string",
                        "description": "a question to ask the QA bot",
                    }
                },
                "required": ["str_or_query_bundle"],
            },
        }
    }
]
tool2=[{
        "type": "function",
        "function": {
            "name": "code_pure_python",
            "description": """
                Give an instruction to a coding assistant to write pure python codes
                """,
            "parameters": {
                "type": "object",
                "properties": {
                    "instruction": {
                        "type": "string",
                        "description": "coding instruction to give to the coding assistant",
                    }
                },
                "required": ["instruction"],
            },
        }
    }
]

tools = [tool1[0], tool2[0]]
funcs = [query_engine.query, code_pure_python]

In [15]:
system = {
    "persona": "A helpful software engineer",
    "requirements": """
        Think step-by-step and provide thoughtful, clear, precise answers. 
        Maintain a humble yet confident tone.
    """,
    "responsibilities": """
        Assist with coding in the lionagi Python package.
    """,
    "tools": """
        Use a QA bot for grounding responses and a coding assistant 
        for writing pure Python code.
    """
}

function_call1 = {
    "notice": """
        Use the QA bot tool at least five times at each task step, 
        identified by the step number. This bot can query source codes 
        with natural language questions and provides natural language 
        answers. Decide when to invoke function calls. You have to ask 
        the bot for clarifications or additional information as needed, 
        up to ten times if necessary.
    """
}

function_call2 = {
    "notice": """
        Use the coding assistant tool at least once at each task step, 
        and again if a previous run failed. This assistant can write 
        and run Python code in a sandbox environment, responding to 
        natural language instructions with 'success' or 'failed'. Provide 
        clear, detailed instructions for AI-based coding assistance. 
    """
}


In [16]:
# Step 1: Understanding User Requirements
instruct1 = {
    "task_step": "1",
    "task_name": "Understand User Requirements",
    "task_objective": "Comprehend user-provided task fully",
    "task_description": """
        Analyze and understand the user's task. Develop plans 
        for approach and delivery. 
    """
}

# Step 2: Proposing a Pure Python Solution
instruct2 = {
    "task_step": "2",
    "task_name": "Propose a Pure Python Solution",
    "task_objective": "Develop a detailed pure Python solution",
    "task_description": """
        Customize the coding task for lionagi package requirements. 
        Use a QA bot for clarifications. Focus on functionalities 
        and coding logic. Add lots more details here for 
        more finetuned specifications
    """,
    "function_call": function_call1
}

# Step 3: Writing Pure Python Code
instruct3 = {
    "task_step": "3",
    "task_name": "Write Pure Python Code",
    "task_objective": "Give detailed instruction to a coding bot",
    "task_description": """
        Instruct the coding assistant to write executable Python code 
        based on improved task understanding. Provide a complete, 
        well-structured script if successful. If failed, rerun, report 
        'Task failed' and the most recent code attempt after a second 
        failure. Please notice that the coding assistant doesn't have 
        any knowledge of the preceding conversation, please give as much 
        details as possible when giving instruction. You cannot just 
        say things like, as previsouly described. You must give detailed
        instruction such that a bot can write it
    """,
    "function_call": function_call2
}


### 4. Workflow and output

In [17]:
# solve a coding task in pure python
async def solve_in_python(context):
    
    # set up session and register both tools to session 
    coder = li.Session(system, dir=output_dir)
    coder.register_tools(tools=tools, funcs=funcs)
    
    # initiate should not use tools
    await coder.initiate(instruct1, context=context, temperature=0.7)
    
    # auto_followup with QA bot tool
    await coder.auto_followup(instruct2, num=num, temperature=0.6, tools=tool1,
                                   tool_parser=lambda x: x.response)
    
    # auto_followup with code interpreter tool
    await coder.auto_followup(instruct3, num=3, temperature=0.5, tools=tool2)
    
    # save to csv
    coder.messages_to_csv()
    coder.log_to_csv()
    
    # return codes
    return coder.conversation.messages[-1]['content']

In [18]:
issue = {
    "raise files and chunks into objects": """
        files and chunks are currently in dict format, please design classes for them, include all 
        members, methods, staticmethods, class methods... if needed. please make sure your work 
        has sufficiednt content, make sure to include typing and docstrings
        """
    }

In [19]:
# this takes 5-15 mins to run
response = await solve_in_python(issue)

user_proxy (to Coder Assistant):

Please write a Python class named `File` with the following attributes: a string `file_name`, a string `file_content`, and an integer `file_size`. Include an initializer that sets these attributes, and a method named `read_content` that simulates reading file content by simply returning the `file_content` attribute.

Next, write a Python class named `Chunk` with the following attributes: a string `chunk_content`, an integer `chunk_id`, a float `chunk_overlap`, an integer `chunk_size`, a float `chunk_threshold`, a `File` instance `file`, an integer `file_chunks`, and an integer `file_size`. The initializer should set these attributes. Also, include a method named `validate_chunk` that checks if the `chunk_size` is greater than or equal to the `chunk_threshold` and returns a boolean result.

Finally, write two functions: `chunk_text` that takes a string, a chunk size, an overlap percentage, and a threshold as arguments and returns a list of `Chunk` insta

In [29]:
import json
Markdown(json.loads(response)['output']['content'])

It seems there was a mistake in the code due to the use of a subscriptable type annotation with `list` which is not supported in older versions of Python. I will correct the code to use the `typing.List` for type annotations. Then I will re-run the code.


The implementation has been corrected and the type annotations have been fixed using `typing.List`. The sample usage shows the successful division of the given file content into chunks. Here is the full implementation:

```python
from typing import List

class File:
    def __init__(self, file_name: str, file_content: str, file_size: int):
        """Initializes a File object with the given name, content, and size."""
        self.file_name = file_name
        self.file_content = file_content
        self.file_size = file_size
    
    def read_content(self) -> str:
        """Simulates reading the file content by returning the file_content attribute."""
        return self.file_content

class Chunk:
    def __init__(self, chunk_content: str, chunk_id: int, chunk_overlap: float,
                 chunk_size: int, chunk_threshold: float, file: File, 
                 file_chunks: int, file_size: int):
        """Initializes a Chunk object with the given attributes."""
        self.chunk_content = chunk_content
        self.chunk_id = chunk_id
        self.chunk_overlap = chunk_overlap
        self.chunk_size = chunk_size
        self.chunk_threshold = chunk_threshold
        self.file = file
        self.file_chunks = file_chunks
        self.file_size = file_size
    
    def validate_chunk(self) -> bool:
        """Checks if the chunk_size is greater than or equal to the chunk_threshold."""
        return self.chunk_size >= self.chunk_threshold

def chunk_text(text: str, chunk_size: int, overlap_percentage: float, 
               threshold: float) -> List[Chunk]:
    """
    Splits the text into chunks of chunk_size, with specified overlap_percentage between chunks.
    Only creates a new chunk if the remaining text meets or exceeds the threshold length.
    Returns a list of Chunk instances representing the divided text.
    """
    chunks = []
    chunk_id = 0
    overlap_size = int(chunk_size * overlap_percentage)
    text_length = len(text)
    file_dummy = File("dummy", text, text_length)
    
    while text_length > threshold:
        end_index = min(chunk_size, text_length)
        chunk = text[:end_index]
        text = text[end_index - overlap_size:]  # Apply overlap for the next chunk
        text_length = len(text)
        chunk_instance = Chunk(
            chunk_content=chunk,
            chunk_id=chunk_id,
            chunk_overlap=overlap_percentage,
            chunk_size=end_index,
            chunk_threshold=threshold,
            file=file_dummy,
            file_chunks=-1,  # Placeholder as we don't know the total number of chunks yet
            file_size=text_length
        )
        chunks.append(chunk_instance)
        chunk_id += 1
    
    # Update the file_chunks attribute
    for chunk_instance in chunks:
        chunk_instance.file_chunks = len(chunks)

    return chunks

def file_to_chunks(file: File, chunk_size: int, threshold: float) -> List[Chunk]:
    """
    Uses the read_content method of the File instance to obtain the file content,
    and divides this content into chunks according to the specified chunk_size and threshold.
    Returns a list of Chunk instances representing the file content divided into chunks.
    """
    content = file.read_content()
    # Assume overlap percentage is given as part of this task, since not specified
    overlap_percentage = 0.0  # No overlap by default
    return chunk_text(content, chunk_size, overlap_percentage, threshold)

# Example usage
sample_file = File("example.txt", "Hello World! This is an example file content.", 36)
chunks = file_to_chunks(sample_file, 10, 5)
[(chunk.chunk_id, chunk.chunk_content) for chunk in chunks]  # Check the generated chunks
```

The sample usage created chunks from a file content as follows:
- Chunk 0: "Hello Worl"
- Chunk 1: "d! This is"
- Chunk 2: " an exampl"
- Chunk 3: "e file con"

TERMINATE


# B. Use Session in other Sessions

-- automatically improve prompts

In [45]:
# this time we want to give the coder a prompt bettering tool, so it can
# instruct the code interpreter more effectively 

output_dir2 = "data/log/prompter/"

system2 = """
    Your primary role is to refine AI bot prompts, ensuring they are clear and 
    effective. Start by understanding the interaction context in detail. Analyze 
    the initial prompts and the bot's responses. When improving a prompt, adopt 
    a step-by-step approach, breaking down complex scenarios into simpler, 
    more manageable tasks. Explain each step of your reasoning for the changes 
    you propose, focusing on how they enhance clarity and effectiveness.
"""

instruct4 = """
    Examine the responses from the AI bot, identifying areas where the original 
    prompts could be causing confusion or inefficiency. When revising, consider 
    how different tools, like query engines or coding bots, are used. For query 
    engines, suggest breaking down complex queries into multiple smaller queries, 
    possibly using parallel function calls for efficiency. For coding bots, provide 
    clear, sequential instructions, ensuring each step is well-defined and 
    understandable. Include specific prompting techniques that can improve the 
    bot's effectiveness. For each modification, offer a comprehensive explanation 
    detailing the rationale behind the change, how it addresses issues in the 
    original prompt, and how it enhances the bot's response quality and relevance.
"""

async def improve_prompts(context):
    prompter = li.Session(system=system2, dir=output_dir2)
    await prompter.initiate(instruction=instruct4, context=context, temperature=0.3)
    
    prompter.messages_to_csv()
    prompter.log_to_csv()
    return prompter.conversation.messages[-1]['content']

In [47]:
tool3 = [
    {
        "type": "function",
        "function": {
            "name": "improve_prompts",
            "description": """
                perform prompts improvements for an AI bot
                """,
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "prompts to improve",
                    }
                },
                "required": ["context"],
            }}}]

tools2 = [tool2[0], tool3[0]]
funcs2 = [code_pure_python, improve_prompts]

In [48]:
function_call3 = {
    "notice": """
        Use 'improve_prompts' with a focus on detail and structure:
        1. Prompt Identification: Highlight the specific part of the prompt that needs 
           refinement. Use line numbers or quote the text directly.
        2. Issue Description: Describe what makes the current prompt unclear or ineffective.
        3. Suggested Improvements: Propose specific changes or enhancements. This could 
           include rephrasing, adding context, or breaking down complex instructions.
        4. Expected Outcome: Explain how your suggested improvements will aid in better 
           understanding or response accuracy from the AI bot.
        5. Test Scenarios: Suggest possible scenarios or questions to test the improved 
           prompts effectiveness.
        By providing these details, you guide the AI bot towards creating more precise, 
        understandable, and effective prompts.
    """
}

instruct3.update({"function_call2": function_call3})

In [50]:
# solve a coding task in pure python
async def solve_issue(context):
    
    coder = li.Session(system, dir=output_dir)
    coder.register_tools(tools=tools2, funcs=funcs2)
    
    await coder.initiate(instruct1, context=context, temperature=0.7)
    await coder.auto_followup(instruct2, num=num, temperature=0.6, tools=tool1,
                                   tool_parser=lambda x: x.response)
    
    # auto_followup with code interpreter tool
    await coder.auto_followup(instruct3, num=5, temperature=0.5, tools=[tool2[0], tool3[0]])
    
    # save to csv
    coder.messages_to_csv()
    coder.log_to_csv()
    
    # return codes
    return coder.conversation.messages[-1]['content']

In [51]:
# this one takes even longer: 15-25 mins
# you can decrease num, the maximum number of tool use allowed per api call, to speed up
response = await solve_issue(issue)

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_Bgu1rE8DlGHap0JDFGtabp60


user_proxy (to Coder Assistant):

Write a Python class named 'File' that represents a file in a system. The class should have the following attributes: 'name' (str) to store the file name, 'size' (int) to store the size of the file in bytes, 'extension' (str) to store the file extension, and 'content' (str) to store the actual content of the file. Include an initializer method that sets these attributes. Also, provide a method named 'read' that returns the content of the file, and a method named 'write' that takes a string as an argument and updates the file's content with it. Ensure all methods have appropriate docstrings explaining their purpose, parameters, and return values.

--------------------------------------------------------------------------------
Coder Assistant (to user_proxy):

Here is the implementation of the `File` class as specified in your request:

```python
class File:
    def __init__(self, name, size, extension, content):
        """
        Initializer for the 

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_XMKRm5rkm2zsNS9bfwO9hqXY


user_proxy (to Coder Assistant):

Write a Python class named 'Chunk' that represents a chunk of data. The class should have the following attributes: 'index' (int) to store the order of the chunk, 'data' (str) to store the actual chunk data, and 'checksum' (str) to store a checksum for data verification. Include an initializer method that sets these attributes. Also, provide a method named 'verify' that checks if the provided checksum matches the computed checksum of the data and returns a boolean value accordingly. Ensure all methods have appropriate docstrings explaining their purpose, parameters, and return values.

--------------------------------------------------------------------------------
Coder Assistant (to user_proxy):

The Python class `Chunk` has been created and tested with example data. Verification of the checksum also returned True, indicating that the class and its methods are functioning correctly. Here is the full implementation:

```python
import hashlib

class Ch

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_FKP0K5KwWuN5iOIQ7BlPwec1


...

user_proxy (to Coder Assistant):

Create a Python class named 'Chunk' with the following specifications:
1. Define an initializer method (__init__) that accepts three parameters: 'index' (int), 'data' (str), and 'checksum' (str). Initialize corresponding instance attributes with these parameters.
2. Add a method named 'verify' that computes the SHA-256 checksum of the 'data' attribute and compares it to the 'checksum' attribute. The method returns True if they match, otherwise False.
3. Include detailed docstrings for the class and each method, explaining their purpose, parameters, and return values. Provide an example in the docstring if necessary.

--------------------------------------------------------------------------------
Coder Assistant (to user_proxy):

The implementation of the `Chunk` class is as follows, and it has been validated with an example:

```python
import hashlib

class Chunk:
    """
    Represents a single chunk of data, with properties to keep track of its
    

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_yvN3qmABlaOT0cUwK9pvHodx


user_proxy (to Coder Assistant):

Create a Python class named 'File' with the following specifications:
1. Define an initializer method (__init__) that accepts four parameters: 'name' (str), 'size' (int), 'extension' (str), 'content' (str). Initialize corresponding instance attributes with these parameters.
2. Add a method named 'read' that takes no parameters and returns the 'content' attribute of the instance.
3. Add a method named 'write' that accepts 'new_content' (str) and updates the 'content' attribute with this new content, also updating the 'size' attribute to the length of the new content.
4. Include detailed docstrings for the class and each method, explaining their purpose, parameters, and return values. Provide an example in the docstring if necessary.

--------------------------------------------------------------------------------
Coder Assistant (to user_proxy):

The `File` class has been implemented successfully and tested. The `read` and `write` methods properly retur

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_Sx8NQLoVVYtnNBJpeeQUDPgR


user_proxy (to Coder Assistant):

Create a Python class named 'Chunk' with the following specifications:
1. Define an initializer method (__init__) that accepts three parameters: 'index' (int), 'data' (str), and 'checksum' (str). Initialize corresponding instance attributes with these parameters.
2. Add a method named 'verify' that computes the SHA-256 checksum of the 'data' attribute and compares it to the 'checksum' attribute. The method returns True if they match, otherwise False.
3. Include detailed docstrings for the class and each method, explaining their purpose, parameters, and return values. Provide an example in the docstring if necessary.

--------------------------------------------------------------------------------
Coder Assistant (to user_proxy):

The `Chunk` class and its `verify` method have been implemented and tested successfully. Here is the complete implementation along with a test that confirms the `verify` method works:

```python
import hashlib

class Chunk:
  

## output

In [ ]:
class File:
    """
    A simple representation of a file that has a name, size, extension, and content.
    
    Attributes:
        name (str): The name of the file.
        size (int): The size of the file content in bytes.
        extension (str): The file extension indicating the file type.
        content (str): The content of the file, represented as a string.
    """
    
    def __init__(self, name, size, extension, content):
        """
        Initializes a new File instance with the provided attributes.
        
        Parameters:
            name (str): The name for the file.
            size (int): The size of the file content in bytes.
            extension (str): The file extension indicating the file type.
            content (str): The initial content of the file.
        """
        self.name = name
        self.size = size
        self.extension = extension
        self.content = content
    
    def read(self):
        """
        Returns the content of the file.
        
        Returns:
            str: The content of the file.
        
        Example:
            file = File("example", 12, ".txt", "Hello World!")
            content = file.read()
            print(content)  # Outputs: Hello World!
        """
        return self.content
    
    def write(self, new_content):
        """
        Updates the content of the file with new content and adjusts the file size.
        
        Parameters:
            new_content (str): The new content to write to the file.
        
        Example:
            file = File("example", 12, ".txt", "Hello World!")
            file.write("Goodbye World!")
            content = file.read()
            print(content)  # Outputs: Goodbye World!
            print(file.size)  # Outputs the size of the new content in bytes
        """
        self.content = new_content
        self.size = len(new_content)

import hashlib

class Chunk:
    """
    A class to represent a chunk of data with its index, content, and checksum.
    
    Attributes
    ----------
    index : int
        The index of the chunk in the sequence of chunks.
    data : str
        The string content of the chunk.
    checksum : str
        The SHA-256 checksum of the data content for verification purposes.
        
    Methods
    -------
    __init__(index: int, data: str, checksum: str)
        Initializes the Chunk with an index, data, and expected checksum.
    verify() -> bool
        Verifies if the actual checksum of the data matches the expected checksum.
    """
    
    def __init__(self, index: int, data: str, checksum: str):
        """
        Constructs all the necessary attributes for the Chunk object.
        
        Parameters
        ----------
        index : int
            The index of the chunk in the sequence of chunks.
        data : str
            The string content of the chunk.
        checksum : str
            The SHA-256 checksum of the data content for verification purposes.
        """
        self.index = index
        self.data = data
        self.checksum = checksum
        
    def verify(self) -> bool:
        """
        Verifies if the actual checksum of the data matches the expected checksum.
        
        Returns
        -------
        bool
            True if the actual and expected checksums match, False otherwise.
        """
        actual_checksum = hashlib.sha256(self.data.encode()).hexdigest()  # Calculate the SHA-256 checksum of the data
        return actual_checksum == self.checksum  # Compare calculated checksum with the expected checksum


# Example Usage:
# Given data and correct checksum calculated using SHA-256 hashing.
chunk_data = "some data"
correct_checksum = hashlib.sha256(chunk_data.encode()).hexdigest()
chunk = Chunk(1, chunk_data, correct_checksum)

# Testing the verify method.
print(chunk.verify())  # Would print 'True' because the checksum matches the data.